# Add harmonic bond force

In [1]:
import molsysmt as msm

In [2]:
import openmm as mm
from openmm import unit
from openmm import app
from tqdm import tqdm
import numpy as np
from matplotlib import pyplot as plt

In [3]:
topology = app.Topology()
chain = topology.addChain('A')
residue = topology.addResidue('Ar', chain)
atom = topology.addAtom(name='Ar', element= app.element.argon, residue=residue)
chain = topology.addChain('B')
residue = topology.addResidue('Ar', chain)
atom = topology.addAtom(name='Ar', element= app.element.argon, residue=residue)

In [4]:
system = mm.System()
system.addParticle(atom.element.mass)
system.addParticle(atom.element.mass)

1

In [5]:
temperature = 300*unit.kelvin

In [6]:
integration_timestep = 2.0*unit.femtoseconds
saving_interval = 1.00*unit.picoseconds
logging_interval = 100.00*unit.picoseconds
simulation_time = 1000.*unit.picoseconds

saving_steps = int(saving_interval/integration_timestep)
logging_steps = int(logging_interval/integration_timestep)
md_steps = int(simulation_time/integration_timestep)

In [7]:
friction   = 5.0/unit.picoseconds
integrator = mm.LangevinIntegrator(temperature, friction, integration_timestep)

In [8]:
platform = mm.Platform.getPlatformByName('OpenCL')

In [9]:
simulation = app.Simulation(topology, system, integrator, platform)

In [10]:
initial_positions  = [[0.0, 0.0, 0.0], [1.0, 1.0, 1.0]] * unit.nanometers
simulation.context.setPositions(initial_positions)

In [11]:
Lbox = 2.0
v1 = [Lbox,0,0] * unit.nanometers
v2 = [0,Lbox,0] * unit.nanometers
v3 = [0,0,Lbox] * unit.nanometers
simulation.context.setPeriodicBoxVectors(v1, v2, v3)

In [12]:
msm.thirds.openmm.forces.add_harmonic_bond_force(simulation, atom_pair=[0,1],
                                                 force_constant='1000 kilojoules/(mol*nanometers**2)',
                                                 bond_length='1 nanometer',
                                                 pbc=True)

0

In [ ]:
reporter_tqdm = msm.thirds.openmm.reporters.TQDMReporter(logging_steps, md_steps, temperature=False)
simulation.reporters.append(reporter_tqdm)

In [ ]:
reporter_trajectory_dict = msm.thirds.openmm.reporters.TrajectoryDictReporter(saving_steps, time=True, coordinates=True)
simulation.reporters.append(reporter_trajectory_dict)

In [ ]:
simulation.step(md_steps)

In [ ]:
trajectory_dict = reporter_trajectory_dict.finalize()

In [ ]:
plt.plot(trajectory_dict['time'], trajectory_dict['coordinates'][:,0,0])
plt.plot(trajectory_dict['time'], trajectory_dict['coordinates'][:,1,0])
plt.axhline(y=0.5, color='red', linestyle='-')
plt.show()

In [ ]:
aa = [[0,1],[2,3]]

In [ ]:
for ii,jj in aa:
    print(ii,jj)